In [1]:
# adds scripts/ and src/ folder: so you can import scripts/functions across project steps
import sys 
sys.path.append("../../src")
sys.path.append("../../scripts")

from os import listdir
from os import path

import pandas as pd
from tqdm.notebook import tqdm

from data_filepaths import image_folders, portraits_csv, images_with_boxes_folder, object_detection_results_csv_folder

In [2]:
#MODEL = "resnet"# model FasterRCNN+InceptionResNet V2: higher accuracy,
MODEL = "mobilenet" # model ssd+mobilenet V2: small and fast.

CASE_STUDY_TITLE = "portraits_before1800"

object_detection_results_csv_folder = object_detection_results_csv_folder.replace("<CASESTUDY>", CASE_STUDY_TITLE).replace("<MODEL>", MODEL)


In [4]:
results_per_image = [
    pd.read_csv(path.join(object_detection_results_csv_folder, csv_path))
    for csv_path in listdir(object_detection_results_csv_folder)
]

In [9]:
# ,detection_class_labels,detection_class_entities,detection_scores,x0,y0,x1,y1,new_filename
relevant_columns = ["detection_class_labels", "detection_class_entities", "detection_scores", "new_filename"]

def unique_detection_classes(result_dtf):
    return result_dtf[relevant_columns].groupby(
        ['detection_class_labels', "detection_class_entities", "new_filename"], as_index=False).agg(
        {'detection_scores':max}
    )

In [10]:
global_uresults = pd.concat([unique_detection_classes(res) for res in results_per_image])

In [11]:
global_uresults

,detection_class_labels,detection_class_entities,new_filename,detection_scores
0,15,b'Human eye',35828.jpg,0.113142
1,69,b'Person',35828.jpg,0.123696
2,99,b'Poster',35828.jpg,0.106311
3,105,b'Tie',35828.jpg,0.092071
4,148,b'Human mouth',35828.jpg,0.103545
...,...,...,...,...
10,433,b'Clothing',78471.jpg,0.491070
11,434,b'Footwear',78471.jpg,0.169724
12,485,b'Window',78471.jpg,0.090742
13,502,b'Human face',78471.jpg,0.150539


In [12]:
nb_portraits_per_object_class = global_uresults.groupby(
        ['detection_class_labels', "detection_class_entities"], as_index=False).agg(
        {'detection_scores':max, "new_filename": "nunique"}
    )
nb_portraits_per_object_class["nb_portraits"] = nb_portraits_per_object_class["new_filename"]
del nb_portraits_per_object_class["new_filename"]
nb_portraits_per_object_class.sort_values(by="nb_portraits", ascending=False, inplace=True)
nb_portraits_per_object_class[0:40]

,detection_class_labels,detection_class_entities,detection_scores,nb_portraits
75,433,b'Clothing',0.732368,347
86,502,b'Human face',0.981114,346
16,69,b'Person',0.551262,320
87,503,b'Human arm',0.218862,312
4,15,b'Human eye',0.164523,304
61,292,b'Human head',0.170420,294
56,253,b'Human hair',0.162239,294
58,277,b'Fashion accessory',0.433286,272
50,228,b'Woman',0.665233,263
35,148,b'Human mouth',0.149974,242


In [13]:
nb_portraits_per_object_class[40:80]

,detection_class_labels,detection_class_entities,detection_scores,nb_portraits
43,194,b'Sun hat',0.137230,9
32,139,b'Cowboy hat',0.231144,8
13,51,b'Boy',0.158164,8
8,29,b'Doll',0.600340,7
33,142,b'Drum',0.102841,7
72,397,b'Trousers',0.096770,7
88,514,b'Jeans',0.154890,6
40,167,b'Scarf',0.148940,5
19,90,b'Necklace',0.089212,5
54,242,b'Horse',0.816975,4
